# Marvel Relationships

Based on the study by Joe Miro and the dataset published at -> http://bioinfo.uib.es/~joemiro/marvel.html

Download the characters.txt. books.txt and edges.txt from the link above.
We will try to map the relationships as a network between characters.



In the edges.txt file, first we have a complete list of all the character and book **vertices** which is followed by a list of **edges** which connects a character with list of books it has appeared in. Multiple rows for a character since a row can have max of 15 book vertices.

In [1]:
spark # initalize

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1515992844769_0008,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.

In [2]:
# Lets read the files now
books = sc.textFile('wasb:///example/data/books.txt')
characters = sc.textFile('wasb:///example/data/characters.txt')
edges= sc.textFile('wasb:///example/data/edges.txt')

In [3]:
books.take(10) # vertex name : book name

[u'Vertex 6487: AA2 35', u'Vertex 6488: M/PRM 35', u'Vertex 6489: M/PRM 36', u'Vertex 6490: M/PRM 37', u'Vertex 6491: WI? 9', u'Vertex 6492: AVF 4', u'Vertex 6493: AVF 5', u'Vertex 6494: H2 251', u'Vertex 6495: H2 252', u'Vertex 6496: COC 1']

In [4]:
characters.take(10) # vertex name: character name

[u'Vertex 1: 24-HOUR MAN/EMMANUEL', u'Vertex 2: 3-D MAN/CHARLES CHANDLER & HAROLD CHANDLER', u'Vertex 3: 4-D MAN/MERCURIO', u'Vertex 4: 8-BALL/', u'Vertex 5: A', u"Vertex 6: A'YIN", u'Vertex 7: ABBOTT, JACK', u'Vertex 8: ABCISSA', u'Vertex 9: ABEL', u'Vertex 10: ABOMINATION/EMIL BLONSKY']

In [5]:
edges.take(10) # At the top this contains only vertices, we need to filter these out and only look for edges

[u'*Vertices 19428 6486', u'1 "24-HOUR MAN/EMMANUEL"', u'2 "3-D MAN/CHARLES CHAN"', u'3 "4-D MAN/MERCURIO"', u'4 "8-BALL/"', u'5 "A"', u'6 "A\'YIN"', u'7 "ABBOTT, JACK"', u'8 "ABCISSA"', u'9 "ABEL"']

In [6]:
# let's write a function which will filter out the vertices from the above RDD
def edgeFilter(row):
    if '*' in row or '"' in row: # first row has a * and subsequent vertice row contain "
        return False
    else:
        return True
    
# let's apply above function
edgesFiltered = edges.filter(edgeFilter)

In [7]:
edgesFiltered.take(10) # this is network representation of a character with the books in the Marvel social universe

[u'1 6487', u'2 6488 6489 6490 6491 6492 6493 6494 6495 6496', u'3 6497 6498 6499 6500 6501 6502 6503 6504 6505', u'4 6506 6507 6508', u'5 6509 6510 6511', u'6 6512 6513 6514 6515', u'7 6516', u'8 6517 6518', u'9 6519 6520', u'10 6521 6522 6523 6524 6525 6526 6527 6528 6529 6530 6531 6532 6533 6534 6535']

### Find the most influential characters

These would be characters which are connected to most number of books.

In [9]:
# split strings into individual
characterBookMap = edgesFiltered.map(lambda x:x.split()).map(lambda x: (x[0], x[1:])) # First element character, rest are books

In [10]:
characterBookMap.take(10)

[(u'1', [u'6487']), (u'2', [u'6488', u'6489', u'6490', u'6491', u'6492', u'6493', u'6494', u'6495', u'6496']), (u'3', [u'6497', u'6498', u'6499', u'6500', u'6501', u'6502', u'6503', u'6504', u'6505']), (u'4', [u'6506', u'6507', u'6508']), (u'5', [u'6509', u'6510', u'6511']), (u'6', [u'6512', u'6513', u'6514', u'6515']), (u'7', [u'6516']), (u'8', [u'6517', u'6518']), (u'9', [u'6519', u'6520']), (u'10', [u'6521', u'6522', u'6523', u'6524', u'6525', u'6526', u'6527', u'6528', u'6529', u'6530', u'6531', u'6532', u'6533', u'6534', u'6535'])]

In [12]:
# We also want to be able to reference the chracter name in the Universe.
# for this we will create a lookup of the chracter and the vertex

def charParse(row):
    row = row.split(":")
    return (row[0][7:], row[1].strip())

characterLookup = characters.map(charParse).collectAsMap() # Pair RDD (tuple) can be collected as a hashtable

In [14]:
characterLookup

{u'5988': u'VALKYRIE II | MUTANT X-VERSE', u'5989': u'VALKYRIE IV', u'5982': u'VAGABOND/PRISCILLA LYONS', u'5983': u'VAGUE/', u'5980': u'UTGARD-LOKI', u'5981': u'VACUUM', u'5986': u'VALINOR', u'5987': u'VALKIN', u'5984': u'VAKUME', u'5985': u'VALERIA', u'6294': u'WOLF SPIRIT/OWAYODATA', u'270': u'AUGUST PERSONAGE IN JADE/YU-TI', u'271': u'AUNTIE FREEZE/', u'272': u'AURA/ANN HERD', u'273': u'AURIC/', u'274': u'AURORA/JEANNE-MARIE BAUBIER', u'275': u'AURORA DOPPELGANGER', u'276': u'AURORA | MUTANT X-VERSE', u'277': u'AUSTIN, GEN.', u'278': u'AUSTIN, SUSAN', u'279': u'AUSTIN, TERRY', u'2143': u'GLADIATRIX', u'2142': u'GLADIATOR II', u'3519': u'MASTER OM', u'3518': u'MASTER OF VENGEANCE', u'3513': u'MASTER MENACE/DR. EMIL BURBANK', u'3512': u'MASTER MAN II/ALEXANDER', u'4641': u'REYES, DR. CECELIA', u'4646': u'RHAPSODY/RACHEL ARGOSY', u'4647': u'RHINO', u'4644': u'REYNOLDS, KATHERINE', u'4645': u'REYNOLDS, LINDY', u'1294': u'CYCLOPS II', u'1295': u'CYCLOPS | MUTANT X-VERSE/SCOTT SUMMERS', 

In [15]:
# calculate the strength of the character
# mapValue() -> special map operation, applied only to the values part of the RDD
# At the end a map function maps the character code with the character name
# Also a reduceByKey operation needs to be applied since the file has multiple values for the same character (15 records per line limit)
characterStrength = characterBookMap.mapValues(lambda x:len(x))\
                                    .map(lambda x: (characterLookup[x[0]], x[1]))\
                                    .reduceByKey(lambda x,y: x+y)\
                                    .sortBy(lambda x:-x[1]) # Sort in the descending order of the books appeared in count

In [16]:
characterStrength.take(10)

[(u'SPIDER-MAN/PETER PARKER', 1625), (u'CAPTAIN AMERICA', 1367), (u'IRON MAN/TONY STARK', 1168), (u'THING/BENJAMIN J. GRIMM', 990), (u'THOR/DR. DONALD BLAKE/SIGURD JARLSON II/JAKE OLSON/LOREN OLSON', 965), (u'HUMAN TORCH/JOHNNY STORM', 908), (u'MR. FANTASTIC/REED RICHARDS', 875), (u'HULK/DR. ROBERT BRUCE BANNER', 841), (u'WOLVERINE/LOGAN', 820), (u'INVISIBLE WOMAN/SUE STORM RICHARDS', 782)]

### Build a co-occurrence network

Co-occurrence network show the relationships between entities (how much they occur together), how many times the Marvel characters have appeared together in our example context. The relationship is stronger if they share a large no of books together.


For our example this will be done in 3 steps.

1. From characterBookMap create a bookCharacterMap (this will have the book id mapping to the characters in it)
2. For each record in the bookCharacterMap, create a list of all the  pairs of the character present in the book.
3. To count no of books in which the pair appears together (stength) is to run reduceByKey() operation

In [48]:
# Step 1 - generate the bookCharacterMap from the characterBookMap RDD.
bookCharacterMap = characterBookMap.flatMapValues(lambda x: x)\
                                    .map(lambda x : (x[1],x[0]))\
                                    .reduceByKey(lambda x,y:x+","+y)\
                                    .mapValues(lambda x:x.split(","))\

In [49]:
bookCharacterMap.take(10)

[(u'11542', [u'545', u'1084', u'1319', u'4415']), (u'11540', [u'545', u'1084', u'1319', u'1347', u'1602', u'3750', u'3959', u'4415', u'5019']), (u'11546', [u'550', u'1077', u'1602', u'3959', u'6316']), (u'11544', [u'548', u'612', u'748', u'865', u'1415', u'2043', u'2239', u'2397', u'2401', u'2545', u'2602', u'2661', u'5920', u'6312', u'6368']), (u'11548', [u'554', u'1869', u'2197', u'2360', u'2776', u'3072', u'3645', u'4613', u'5121', u'5662', u'5954', u'6399']), (u'19399', [u'6372']), (u'19397', [u'6351']), (u'19395', [u'6351']), (u'19393', [u'6316']), (u'19391', [u'6313'])]

In [59]:
import itertools
cooccurenceMap = bookCharacterMap.flatMap(lambda x: list(itertools.combinations(x[1], 2)))

In [60]:
cooccurenceMap.take(10)

[(u'545', u'1084'), (u'545', u'1319'), (u'545', u'4415'), (u'1084', u'1319'), (u'1084', u'4415'), (u'1319', u'4415'), (u'545', u'1084'), (u'545', u'1319'), (u'545', u'1347'), (u'545', u'1602')]

In [70]:
import itertools # note that the above two cells using the same exact code generate a different coccurrence map, which is why
# I have captured this here. Because the data is in-consistent. Below is what I observed
# I am seeing two entries for a pair e.g. (Captain America , Tony Start , 256) and (Tony Stark, Captain America , 229)
# But if the output matches above two, a single entry is show for the pair. (Captain America, Tony Star, (229 +256)).
testmap = bookCharacterMap.flatMap(lambda x: list(itertools.combinations(x[1], 2)))
testmap.take(10)

[(u'4415', u'545'), (u'4415', u'1084'), (u'4415', u'1319'), (u'545', u'1084'), (u'545', u'1319'), (u'1084', u'1319'), (u'1602', u'3750'), (u'1602', u'3959'), (u'1602', u'4415'), (u'1602', u'5019')]

In [69]:
cooccurrenceStrength = cooccurrenceMap.map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)
cooccurrenceStrength.take(10)

[((u'2558', u'3757'), 3), ((u'3770', u'4483'), 1), ((u'4621', u'6452'), 1), ((u'5164', u'6000'), 2), ((u'1567', u'5705'), 1), ((u'4860', u'6242'), 1), ((u'3987', u'5978'), 2), ((u'1948', u'4288'), 3), ((u'1286', u'6166'), 2), ((u'4726', u'4984'), 2)]

In [62]:
# now let's create edges
cooccurrenceEdges = cooccurrenceStrength.map(lambda x: (x[0][0], x[0][1], x[1]))
cooccurrenceEdges.take(10)

[(u'2558', u'3757', 3), (u'3770', u'4483', 1), (u'4621', u'6452', 1), (u'5164', u'6000', 2), (u'1567', u'5705', 1), (u'4860', u'6242', 1), (u'3987', u'5978', 2), (u'1948', u'4288', 3), (u'1286', u'6166', 2), (u'4726', u'4984', 2)]

### Find the most important relationships

Understanding and modeling relationships.
Let's find out top 10 most important relationships in the Marvel universe.

In [63]:
# First sort it based on the edge wieght (descending)
# map do the character name lookups

sortedCooccurrence = cooccurrenceEdges.sortBy(lambda x: -x[2]).map(lambda x: (characterLookup[x[0]], characterLookup[x[1]], x[2]))

In [64]:
sortedCooccurrence.take(10)

[(u'HUMAN TORCH/JOHNNY STORM', u'THING/BENJAMIN J. GRIMM', 744), (u'HUMAN TORCH/JOHNNY STORM', u'MR. FANTASTIC/REED RICHARDS', 713), (u'MR. FANTASTIC/REED RICHARDS', u'THING/BENJAMIN J. GRIMM', 708), (u'INVISIBLE WOMAN/SUE STORM RICHARDS', u'MR. FANTASTIC/REED RICHARDS', 701), (u'HUMAN TORCH/JOHNNY STORM', u'INVISIBLE WOMAN/SUE STORM RICHARDS', 694), (u'INVISIBLE WOMAN/SUE STORM RICHARDS', u'THING/BENJAMIN J. GRIMM', 668), (u'SPIDER-MAN/PETER PARKER', u'WATSON-PARKER, MARY JANE', 616), (u'JAMESON, J. JONAH', u'SPIDER-MAN/PETER PARKER', 526), (u'CAPTAIN AMERICA', u'IRON MAN/TONY STARK', 446), (u'SCARLET WITCH/WANDA MAXIMOFF', u'VISION', 422)]

In [72]:
# Find out strongest relationship for a particular character
sortedCooccurrence.filter(lambda x: 'IRON MAN/TONY STARK' in x).take(10)

[(u'CAPTAIN AMERICA', u'IRON MAN/TONY STARK', 446), (u'IRON MAN/TONY STARK', u'SCARLET WITCH/WANDA MAXIMOFF', 372), (u'IRON MAN/TONY STARK', u'THOR/DR. DONALD BLAKE/SIGURD JARLSON II/JAKE OLSON/LOREN OLSON', 344), (u'IRON MAN/TONY STARK', u'VISION', 333), (u'IRON MAN/TONY STARK', u'WASP/JANET VAN DYNE PYM', 301), (u'HAWK', u'IRON MAN/TONY STARK', 294), (u'ANT-MAN/DR. HENRY J. PYM', u'IRON MAN/TONY STARK', 291), (u'IRON MAN/TONY STARK', u'WONDER MAN/SIMON WILLIAMS', 249), (u'IRON MAN/TONY STARK', u'IRON MAN IV/JAMES R. JIM RHODEY RHODES', 189), (u'IRON MAN/TONY STARK', u'JARVIS, EDWIN', 183)]

In [71]:
# Find out strongest relationship for a particular character
sortedCooccurrence.filter(lambda x: 'SPIDER-MAN/PETER PARKER' in x).take(10)

[(u'SPIDER-MAN/PETER PARKER', u'WATSON-PARKER, MARY JANE', 616), (u'JAMESON, J. JONAH', u'SPIDER-MAN/PETER PARKER', 526), (u'PARKER, MAY', u'SPIDER-MAN/PETER PARKER', 380), (u'ROBERTSON, JOE', u'SPIDER-MAN/PETER PARKER', 356), (u'LEEDS, BETTY BRANT', u'SPIDER-MAN/PETER PARKER', 249), (u'SPIDER-MAN/PETER PARKER', u'THOMPSON, EUGENE FLASH', 240), (u'OSBORN, HARRY', u'SPIDER-MAN/PETER PARKER', 181), (u'HUMAN TORCH/JOHNNY STORM', u'SPIDER-MAN/PETER PARKER', 147), (u'CAPTAIN AMERICA', u'SPIDER-MAN/PETER PARKER', 145), (u'OSBORN, LIZ ALLAN', u'SPIDER-MAN/PETER PARKER', 138)]

In [65]:
sortedCooccurrence.filter(lambda x: 'CAPTAIN AMERICA' in x).take(10)

[(u'CAPTAIN AMERICA', u'IRON MAN/TONY STARK', 446), (u'CAPTAIN AMERICA', u'THOR/DR. DONALD BLAKE/SIGURD JARLSON II/JAKE OLSON/LOREN OLSON', 386), (u'CAPTAIN AMERICA', u'VISION', 385), (u'CAPTAIN AMERICA', u'WASP/JANET VAN DYNE PYM', 384), (u'CAPTAIN AMERICA', u'SCARLET WITCH/WANDA MAXIMOFF', 374), (u'CAPTAIN AMERICA', u'HAWK', 322), (u'ANT-MAN/DR. HENRY J. PYM', u'CAPTAIN AMERICA', 295), (u'CAPTAIN AMERICA', u'JARVIS, EDWIN', 244), (u'CAPTAIN AMERICA', u'WONDER MAN/SIMON WILLIAMS', 216), (u'CAPTAIN AMERICA', u'FALCON/SAM WILSON', 189)]